# TS-EDA — Lag / Rolling / Leakage
- 目标序列行为（趋势/波动/结构变化）
- 自相关与可预测性线索（lag）
- 非平稳性（rolling mean/std）
- **泄露检查（shift-before-rolling）**
- 严格时间切分 sanity check
- 简单 baseline（lag-1）验证

---
## 使用说明
把下面三处变量按你的数据改掉即可：
- `time_col`：时间列名（如果没有就设为 `None`）
- `target_col`：目标列名（如果 y 单独给出，可跳过）
- `df`：你的 DataFrame


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 可选：如果你想用 seaborn，把下面两行取消注释
# import seaborn as sns
# sns.set_style("whitegrid")

# ====== TODO: 按你的数据改这里 ======
time_col = None          # e.g. "timestamp"；如果没有时间列，设为 None
target_col = "y"         # e.g. "target"
# df = ...               # 你的 DataFrame
# ====================================

## 1) 基础检查：时间排序 + y 定义

**关键**：时序分析前先确保按时间排序（如果有时间列）。

In [ ]:
# 如果有时间列：排序并设置索引（可选）
if time_col is not None:
    df = df.sort_values(time_col).reset_index(drop=True)

# 取出 y
if target_col in df.columns:
    y = df[target_col].astype(float)
else:
    raise ValueError(f"target_col={target_col} not found in df.columns")

print("n_rows:", len(df))
print("n_features:", df.shape[1])
print("y summary:\n", y.describe())

## 2) 目标序列概览：趋势/波动/结构变化

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(y.values)
plt.title("Target Time Series (y)")
plt.xlabel("Time index")
plt.ylabel("y")
plt.show()

## 3) 自相关线索：y vs lag(y)

如果 `y(t)` 与 `y(t-1)` 明显相关，说明 **lag 特征很可能有用**。

In [ ]:
y_lag1 = y.shift(1)

plt.figure(figsize=(5,5))
plt.scatter(y_lag1, y, alpha=0.3)
plt.xlabel("y(t-1)")
plt.ylabel("y(t)")
plt.title("Lag-1 Relationship")
plt.show()

print("corr(y, y_lag1) =", y.corr(y_lag1))

## 4) 多个 lag 的相关性

In [ ]:
lags = [1,2,3,5,10,20,50]
out = []
for lag in lags:
    out.append((lag, y.corr(y.shift(lag))))
pd.DataFrame(out, columns=["lag", "corr"]).sort_values("lag")

## 5) 非平稳性检查：rolling mean / rolling std

如果 rolling mean 或 std 随时间明显变化，说明序列可能不严格平稳（stationary）。

In [ ]:
window = 50  # 可调：小数据用 20，大数据用 100+

rolling_mean = y.rolling(window).mean()
rolling_std  = y.rolling(window).std()

plt.figure(figsize=(10,4))
plt.plot(y.values, alpha=0.4, label="y")
plt.plot(rolling_mean.values, label=f"rolling_mean({window})")
plt.plot(rolling_std.values, label=f"rolling_std({window})")
plt.legend()
plt.title("Rolling Statistics")
plt.xlabel("Time index")
plt.show()

## 6) Rolling 特征的泄露检查（最关键）

- ❌ **错误**：`y.rolling(k).mean()` 会包含当前时刻 `y(t)`  
- ✅ **正确**：先 `shift(1)` 再 rolling：`y.shift(1).rolling(k).mean()`

下面用散点图直观看 rolling feature 是否可能对 y 有解释力。

In [ ]:
k = 10  # rolling 窗口
roll_mean_past_k = y.shift(1).rolling(k).mean()   # ✅ only past info

plt.figure(figsize=(5,5))
plt.scatter(roll_mean_past_k, y, alpha=0.3)
plt.xlabel(f"rolling_mean_past_{k} (shifted)")
plt.ylabel("y(t)")
plt.title("Rolling Feature (past-only) vs y")
plt.show()

print("corr(y, roll_mean_past_k) =", y.corr(roll_mean_past_k))

## 7) 时间切分 sanity check（禁止 random split）

**Random split 会把未来信息带进训练**。
这里画出 train/test 在时间轴上的位置，确保是 chronological split。

In [ ]:
split_frac = 0.8
split_idx = int(split_frac * len(df))

plt.figure(figsize=(10,4))
plt.plot(y.iloc[:split_idx].values, label="Train")
plt.plot(np.arange(split_idx, len(df)), y.iloc[split_idx:].values, label="Test")
plt.axvline(split_idx, linestyle="--")
plt.legend()
plt.title(f"Chronological Split (train={split_frac:.0%})")
plt.xlabel("Time index")
plt.show()

## 8) 快速 baseline（lag-1）验证：是否存在可预测性

这一步特别好用：你能在 EDA 阶段就说——**“连 lag-1 baseline 都有点效果，说明有信号”**。

这里用残差分布做直观检查。

In [ ]:
baseline_pred = y.shift(1)
resid = (y - baseline_pred).dropna()

plt.figure(figsize=(10,4))
plt.hist(resid.values, bins=50)
plt.title("Residuals of Lag-1 Baseline: y(t) - y(t-1)")
plt.xlabel("residual")
plt.ylabel("count")
plt.show()

print("resid mean:", resid.mean())
print("resid std :", resid.std())

## 9) 总结
- 目标序列呈现明显的时间依赖（autocorrelation），所以不能当 i.i.d. 数据处理  
- Lag 特征与 Rolling（past-only）特征很可能有效  
- rolling mean/std 随时间变化 → 可能非平稳，需要谨慎做全局标准化  
- 特征构造必须 **shift-before-rolling**，避免 target leakage  
- 训练/测试切分必须按时间顺序（chronological split）  
- Lag-1 baseline 残差表明存在一定可预测性，可作为后续模型对比基准
